# Trabalho Individual 2 

## Alínea f)

## Implementação do Algoritmo Simulated Annealing

In [2]:
#Bibliotecas
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
import random
random.seed(456)
import numpy as np
import math

In [99]:
# Leitura do ficheiro excel que contém a tabela dos rendimentos e índices de impacto ambiental
tabela = pd.read_excel('tabela zonas.xlsx', names=['zona', 'rentabilidade esperada', 'impacto ambiental'])
nodes = len(tabela)  # Obtém o número de zonas

# Leitura da matriz de adjacências
matriz_adj = pd.read_excel('adjacencias.xlsx', names=['zona', 'zonas adjacentes'])

# Parâmetros do problema:
M = [5, 5, 5, 5, 5]  # Número de iterações para cada temperatura
n_temperaturas = len(M) 
alpha = 0.5  # Fator de resfriamento
#T0 = 0.2*calcular_custo(S,tabela)  --> Temperatura inicial

#As soluções exploradas neste algoritmo são guardadas numa data frame que guarda informação sobre a iteração, a temperatura, o circuito e respetivo custo, a probabilidade de aceitação e a aceitação (ou não) da solução
all_sols=pd.DataFrame(columns=['iter', 'temp', 'zonas', 'rendimento', 'prob_aceitação', 'aceitação'])
contador=0
all_sols.loc[contador,:]={'iter': 0, 'temp': '-', 'zonas': '-', 'rendimento': 0, 'prob_aceitação': '-', 'açeitação': '-'}
contador+=1

# Função para calcular a probabilidade de aceitação e aceitar a solução vizinha com base na variação de custo e na temperatura
def accept_solution(delta, temperature):
    if delta < 0:
        return True
    else:
        prob_aceitacao = np.exp(-delta / temperature)
        return random.uniform(0.0, 1.0) < prob_aceitacao
        
#Função para determinar uma solução admissível aleatória 
def gerar_solucao_admissivel(matriz_adj, tabela):
    # Transformar as zonas e suas adjacências em um dicionário para acesso rápido
    adj_dict = matriz_adj.set_index('zona')['zonas adjacentes'].to_dict()
    for key, value in adj_dict.items():
        adj_dict[key] = value.split(', ')  # Supondo que as zonas adjacentes estão separadas por vírgulas e espaços
    
    # Transformar a tabela em um dicionário para acesso rápido
    impacto_dict = tabela.set_index('zona')['impacto ambiental'].to_dict()
    
    # Inicializa uma lista para armazenar as zonas selecionadas
    zonas_selecionadas = []
    
    # Inicializa a soma do impacto ambiental
    soma_impacto = 0
    
    while len(zonas_selecionadas) < 4:
        # Escolhe aleatoriamente uma zona
        zona = random.choice(list(adj_dict.keys()))
        
        # Verifica se a zona já foi selecionada ou se alguma de suas adjacências já foi selecionada
        if zona not in zonas_selecionadas and not any(adj in zonas_selecionadas for adj in adj_dict[zona]):
            # Calcula o novo impacto ambiental se essa zona fosse adicionada
            novo_impacto = soma_impacto + impacto_dict[zona]
            
            # Verifica a soma dos índices de impacto ambiental com base na quantidade de zonas selecionadas
            if (len(zonas_selecionadas) == 0 and novo_impacto <= 6) or \
               (len(zonas_selecionadas) == 1 and novo_impacto <= 7) or \
               (len(zonas_selecionadas) == 2 and novo_impacto <= 8) or \
               (len(zonas_selecionadas) == 3 and novo_impacto <= 8):
                # Adiciona a zona à lista de zonas selecionadas e atualiza a soma do impacto ambiental
                zonas_selecionadas.append(zona)
                soma_impacto = novo_impacto

    return zonas_selecionadas
    
# Função para determinar uma solução vizinha com base na estrutura de vizinhança utilizada na alínea e)
def gerar_solucao_vizinha(solucao, matriz_adj, tabela):
    # Transformar as zonas e suas adjacências em um dicionário para acesso rápido
    adj_dict = matriz_adj.set_index('zona')['zonas adjacentes'].to_dict()
    for key, value in adj_dict.items():
        adj_dict[key] = value.split(', ')  # Supondo que as zonas adjacentes estão separadas por vírgulas e espaços
    
    # Transformar a tabela em um dicionário para acesso rápido
    impacto_dict = tabela.set_index('zona')['impacto ambiental'].to_dict()

    # Remover a última zona da solução atual
    nova_solucao = solucao[:-1]
    impacto_atual = sum(impacto_dict[z] for z in nova_solucao)
    
    while True:
        # Escolhe aleatoriamente uma zona
        zona = random.choice(list(adj_dict.keys()))
        
        # Verifica se a zona já foi selecionada ou se é adjacente a alguma zona já selecionada
        if zona not in nova_solucao and not any(adjacente in nova_solucao for adjacente in adj_dict[zona]):
            # Verifica a soma dos índices de impacto ambiental
            novo_impacto = impacto_atual + impacto_dict[zona]
            if novo_impacto <= 8:
                nova_solucao.append(zona)
                break
    
    return nova_solucao

# Função para calcular o custo de uma solução
def calcular_custo(solucao, tabela):
    custo_total = 0
    for zona in solucao:
        custo_total += tabela.loc[tabela['zona'].astype(str) == zona, 'rentabilidade esperada'].values[0]
    return custo_total

# Função para calcular o impacto ambiental total de uma solução
def calcular_impacto(solucao, tabela):
    impacto_tot = 0
    for zona in solucao:
        impacto_tot += tabela.loc[tabela['zona'].astype(str) == zona, 'impacto ambiental'].values[0]
    return impacto_tot

solucao_inicial = gerar_solucao_admissivel(matriz_adj, tabela) #Solução inicial gerada aleatoriamente
S = solucao_inicial
custo_atual = calcular_custo(S, tabela)  #Corresponde ao custo da solução inicial

##Aplicação do Simulated Annealing

#1. Inicializar a temperatura T0, o fator de resfriamento alpha e o número máximo de iterações max_iter
T0 = 0.2*custo_atual  #Temperatura inicial
temperatura = T0
alpha  # Fator de resfriamento
max_iter = n_temperaturas  # Número máximo de iterações

# 2. Inicializar a solução S 
melhor_solucao = S 

# 3. Calcular o custo inicial
melhor_custo = custo_atual

# 4. Iteração principal do Simulated Annealing
for k in range(n_temperaturas):
    Tk = 0.5 ** k * T0  # Definir a temperatura Tk
    for _ in range(max_iter):
        # Gerar uma solução vizinha S'
        S_vizinha = gerar_solucao_vizinha(S,matriz_adj,tabela)
        
        # Calcular o custo da solução vizinha
        custo_vizinho = calcular_custo(S_vizinha,tabela)
        #custo_vizinho = 10400 
        
        # Calcular a variação de custo delta_f (entre o valor da nova solução e o valor da solução atual)
        delta = custo_vizinho - custo_atual

        # Aceitar a solução vizinha com base na variação de custo e temperatura atual Tk
        aceita = accept_solution(delta, temperatura)
        if aceita:
            # Atualizar a solução atual, caso a solução vizinha seja aceite
            S = S_vizinha
            custo_atual = custo_vizinho
        
        # Escrita das informações da iteração
        all_sols.loc[contador] = {'iter': k, 'temp': temperatura, 'zonas': S, 'rendimento': custo_vizinho, 'prob_aceitação': np.exp(-delta / temperatura), 'aceitação': aceita}
        contador += 1
            
        # Atualizar a melhor solução encontrada (se necessário)
        if custo_vizinho > melhor_custo:
            melhor_solucao = S_vizinha
            melhor_custo = custo_vizinho
        #Calcular impacto ambiental total da melhor solução encontrada
        impacto_total = calcular_impacto(melhor_solucao, tabela)    
    # Reduzir a temperatura Tk+1
    temperatura *= alpha 

#5. Retornar a solução S encontrada
with open("output_sa.txt", "w") as f:
    print('Resumo da implementação do Simulated Annealing:', file=f)        
    print(all_sols, file=f)        
    print('Melhor solução:', melhor_solucao, file=f)
    print('Rendimento total esperado:',melhor_custo, file=f)
    print('Impacto ambiental total:',impacto_total, file=f)

print('Solução inicial:',solucao_inicial)
print('Melhor solução:',melhor_solucao)
print('Rendimento total esperado:',melhor_custo)
print('Impacto ambiental total:',impacto_total)

Solução inicial: ['A20', 'A8', 'A22', 'A28']
Melhor solução: ['A20', 'A8', 'A22', 'A14']
Rendimento total esperado: 15900
Impacto ambiental total: 8
